In [ ]:
#imports
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
#import tensorflow_datasets as tfds
from tensorflow import keras
import pandas as pd
import os
import cv2
import keras.layers as kl
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

from keras.layers import Input, Lambda, Dense, Flatten,Dropout
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

#number of samples that will be passed through to the network at one time
#"Generally batch size of 32 or 25 is good, with epochs = 100 unless you have large dataset"
batch_size = 20
#img_height = 300
#img_width = 300
#import image directories
#data_dir = '/Users/laure/Desktop/melanoma/melanoma_cancer_dataset2'

#label benign images with 0, malignant images as 1

class_names = ['benign', 'malignant']

class_names_label = {class_name: i for i, class_name in enumerate(class_names)}

num_classes = len(class_names)

IMAGE_SIZE = 150

#function to load data
def load_data():
    DIRECTORY = '/Users/15044/Desktop/melanoma_cancer_dataset'
    CATEGORY = ['test', 'train']

    output = []

    for category in CATEGORY:
        print("loading category " + category + "\n")

        path = os.path.join(DIRECTORY, category)
        #path is now set to /test or /train
        images = []
        labels = []

        for folder in os.listdir(path):
            label = class_names_label[folder]   
            #i.e, recall that label is 0 for benign, 1 for malignant 
            print("in folder " + folder+"\n")

            #iterate through each image in folder
            for file in os.listdir(os.path.join(path, folder)):
                #path is now /test/malignant or other combination od test, train, malignant, benign
                
                #create img path
                img_path = os.path.join(path, folder, file)
                #path is now, e.g. /test/benign/image47.jpg

                #open image and resize
                    #read image
                image = cv2.imread(img_path)
                    #convert to RGB color from cv2 default color value byte system
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #resize image to desired size
                image = cv2.resize(image, [IMAGE_SIZE, IMAGE_SIZE])

                #append image and its label
                    #image is appended as its array representation, label is benign or malignant
                images.append(image)
                labels.append(label)
                #print("appended\n")
        #convert to numpy array
        images = np.array(images, dtype='float32') 
        labels = np.array(labels, dtype = 'int32') #can use int because working with 0 and 1 values

        #the output will be a list of images and their respective label in tuple form
        output.append((images, labels))

    #return the output list
    return output

#call load_data() function to create train and test dataset
(train_images, train_labels), (test_images, test_labels) = load_data()

print("data loaded\n")

#shuffling train images and labels trains the model better
train_images, train_labels = shuffle(train_images, train_labels, random_state = 25)

#standardize/normalize the data
#rescale data from RGB (0, 255) to B/W (0, 1)
#this is because we want smaller data values to make the data easier to work with and easier for neural net to pick features out
#this is done in the model below in k1.Rescaling

#making the modes (using Keras sequential model)
#copied and pasted this just as a starting point
model = Sequential([
  # the 3s refer to 3 colors (RGB)
  # relu

  #kl.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  #2D refers to image
  #using relu activation function (default)
  # params:
    # Conv2D( number of feature options that the NN is looking for, ( window length and width if when the NN is taking "steps" through the image ) )
      # the "window" size AKA kernel size is the "art": mess around with it
  #kl.Conv2D(256, (3, 3), padding='same', activation='relu'),
  #kl.Conv2D(256, (3, 3), padding='same', activation='relu'),
  #kl.Conv2D(128, (3, 3), padding='same', activation='relu'),
  kl.Conv2D(16, (3, 3), padding='same', activation='relu'),
  kl.Conv2D(32, (3, 3), padding='same', activation='relu'),
  kl.MaxPooling2D(2, 2),
  #hidden layers
  kl.Conv2D(32, (3, 3), padding='same', activation='relu'),
  kl.MaxPooling2D(2, 2),
  kl.Conv2D(64, (3, 3), padding='same', activation='relu'),
  kl.MaxPooling2D(2, 2),
  kl.Conv2D(128, (3, 3), padding='same', activation='relu'),
  kl.MaxPooling2D(2, 2),
  #flatten outputs to reduce number of features
  kl.Flatten(),
  #output layers
  kl.Dense(128, activation='relu'),
  kl.Dense(num_classes)
])

#compiling the model
#copied and pasted this just as a starting point
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print("model compiled\n")

#adam optimizer: stoichastic gradient descent, default
# loss: 
# "everybody looks at accuracy"



#training the model 
#epoch: one complete pass through training data 


history = model.fit(train_images, train_labels, batch_size = 20, epochs = 100, validation_split=.2)
print('model has been trained\n')

print(model.summary())


#check with test images
print("testing model accuracy on test image and label set")
test_loss = model.evaluate(test_images, test_labels)

#this is the only part i dont understand yet but we'll get there i just wanted to see what it did
predictions = model.predict(test_images)    #vector of probabilities
pred_labels = np.argmax(predictions, axis=1) #take highest probability

print(classification_report(test_labels, pred_labels))

#print confusion matrix

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

cm = confusion_matrix(test_labels, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix = cm)

disp.plot()
plt.show()

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.ylim(0, 1)
plt.show 

loading category test

in folder benign

in folder malignant

loading category train

in folder benign

in folder malignant

data loaded

model compiled

Epoch 1/100
40/40 [==============================] - 22s 482ms/step - loss: 8.5359 - accuracy: 0.6737 - val_loss: 0.3908 - val_accuracy: 0.8150
Epoch 2/100
40/40 [==============================] - 18s 449ms/step - loss: 0.3742 - accuracy: 0.8175 - val_loss: 0.3641 - val_accuracy: 0.8500
Epoch 3/100
40/40 [==============================] - 18s 451ms/step - loss: 0.3684 - accuracy: 0.8425 - val_loss: 0.3264 - val_accuracy: 0.8500
Epoch 4/100
40/40 [==============================] - 18s 447ms/step - loss: 0.3071 - accuracy: 0.8637 - val_loss: 0.3592 - val_accuracy: 0.8600
Epoch 5/100
40/40 [==============================] - 18s 453ms/step - loss: 0.2848 - accuracy: 0.8737 - val_loss: 0.3094 - val_accuracy: 0.8400
Epoch 6/100
40/40 [==============================] - 18s 448ms/step - loss: 0.2905 - accuracy: 0.8825 - val_loss: 0.4777 - val

40/40 [==============================] - 18s 460ms/step - loss: 0.1427 - accuracy: 0.9538 - val_loss: 0.3549 - val_accuracy: 0.8800
Epoch 57/100
40/40 [==============================] - 18s 456ms/step - loss: 0.1381 - accuracy: 0.9500 - val_loss: 0.3882 - val_accuracy: 0.8750
Epoch 58/100
40/40 [==============================] - 18s 456ms/step - loss: 0.1441 - accuracy: 0.9488 - val_loss: 0.5336 - val_accuracy: 0.8650
Epoch 59/100
40/40 [==============================] - 18s 452ms/step - loss: 0.1527 - accuracy: 0.9413 - val_loss: 0.5311 - val_accuracy: 0.8550
Epoch 60/100
40/40 [==============================] - 18s 452ms/step - loss: 0.1409 - accuracy: 0.9475 - val_loss: 0.3275 - val_accuracy: 0.8900
Epoch 61/100
40/40 [==============================] - 18s 458ms/step - loss: 0.1374 - accuracy: 0.9513 - val_loss: 0.5206 - val_accuracy: 0.8300
Epoch 62/100
40/40 [==============================] - 18s 448ms/step - loss: 0.5011 - accuracy: 0.8125 - val_loss: 0.3486 - val_accuracy: 0.885